In [1]:
import numpy as np
import opendssdirect as dss
from utils.device.Inverter import Inverter
from utils.controller.AdaptiveInvController import AdaptiveInvController
from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

%matplotlib inline

# import numpy as np
# from matplotlib.mlab import PCA
from sklearn.decomposition import PCA

import scipy.linalg as spla


#######################################################
#######################################################
###Global variable initialization and error checking###
#######################################################
#######################################################
Sbase=1
LoadScalingFactor = 3
GenerationScalingFactor = 5
SlackBusVoltage = 1.00
NoiseMultiplyer= 1
#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40600
EndTime = StartTime + 640
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 160
PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Set initial VBP parameters for uncompromised inverters
VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

#Set delays for each node
Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
#Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
lpf_meas_vector = (1+0.2*np.random.randn(31))
lpf_output_vector= (0.1+0.015*np.random.randn(31))

#Set observer voltage threshold
ThreshHold_vqvp = 0.06
adaptive_gain=800


power_factor=0.9
pf_converted=tan(acos(power_factor))
# Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters


#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Inappropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')
    
dss.run_command('Redirect feeder/03node_singlephase_balanced.dss')
# dss.run_command('Redirect feeder/feeder34_B_NR/network_05node_3phase_radial.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')
print(TotalLoads)
# dir(dss.Loads)

# (dss.Loads.AllNames())

# [x for x in dss.Circuit.AllBusNames() if 'bus' in x ]

# dss.Circuit.AllBusNames()

# dir(dss.Loads.AllNames)
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

print(dss.Bus.kVBase())

for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         print(dss.Bus.kVBase())
        tempVoltage = dss.Bus.VMagAngle()
        tempVoltage = tempVoltage
    #     print(tempVoltage)
#         print(tempVoltage[::2])
#         print(tempVoltage[0]*np.sqrt(3)/1)
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

    
# dir(dss.Bus)

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

2
2
['s701a', 's702a']
5
['sourcebus', '799', '799r', '701', '702']
132.79056191361394
sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132789.73307286427, -0.0005618634830067374, 132790.48362023226, -120.00001465921365, 132790.6305220668, 119.9999780835622]
puVMAGANGLE - LN POLAR
[0.9999937582856966, -0.0005618634830067374, 0.9999994103995002, -120.00001465921365, 1.0000005166666355, 119.9999780835622]
VOTLAGES - LN CARTESIAN
[132789.7330664794, -1.302184952691617, -66395.2712459576, -114999.91520114649, -66395.27128482924, 115000.08480632518]
puVOTLAGES - LN CARTESIAN
[0.9999937582376144, -9.806306517015458e-06, -0.49999992687093703, -0.8660247651934704, -0.499999927163666, 0.8660260424316735]
VLL - LL CARTESIAN
[199185.00431243703, 114998.6130161938, 3.887164348270744e-05, -230000.00000747165, -199185.00435130866, 115001.38699127786]
puVLL - LL CARTESIAN
[0.866021757880161, 0.4999939696356

In [ ]:
A = np.random.randn(2,2)
A = np.array([[0, 1],[-2, -1]])

x = np.random.randn(2,8)
y = A@x

print(A)
print(x)
print(y)

# print(y.T)
# print(y.T.reshape((-1, 1), order="F"))

yls = y.T.reshape((-1, 1), order="F")
xls = spla.block_diag(x.T,x.T)

print(xls)

Bvec = spla.inv(xls.T@xls)@(xls.T@yls)

print(Bvec)

B = Bvec.reshape((2,2), order='F')
print(B)
A = B.T
print(A)

In [ ]:

Ts = 0.1

m = 1
b = 0.2
k = 10

Ac = np.array([[0, 1],[-k/m, -b/m]])

Ad = spla.expm(Ac*Ts)
Ad = np.eye(2) + Ts*Ac + 1/2*Ts**2*Ac@Ac


print(Ad)

evals, evecs = spla.eig(Ad)
print(evals)
print(np.abs(evals))

time = Ts*np.arange(0,1001)

x = np.zeros((2,len(time)))
x[:,0] = [0.2, 0]

# print(len(time))
# print(x.shape)

for kt in range(len(time)-1):
    
    x[:,[kt+1]] = Ad@x[:,[kt]]
    
fig = plt.figure(figsize=[12,6])
plt.subplot(2,1,1)
plt.plot(time,x[0,:])
# plt.title('Node Active Power Injection')
# # plt.xlabel('Time')
# plt.ylabel('Active Power Injection [kW]')
plt.subplot(2,1,2)
plt.plot(time,x[1,:])
# plt.plot(time/3600,QinjTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.title('Node Reactive Power Injection')
plt.xlabel('Time')
# plt.ylabel('Reactive Power Injection [kVAr]')
plt.show()


Ns = 2

for kt in range(0,5):
    
    xx = x[:,kt:kt+Ns]
#     print(xx)
    
    yy = x[:,kt+1:kt+1+Ns]
#     print(yy)

    xxblkdiag = spla.block_diag(xx.T,xx.T)
#     for k1 in range(0,1):
#         xxblkdiag = spla.block_diag(xxblkdiag,xx.T)
    print(xxblkdiag.shape)

    yyvec = yy.T.reshape((-1,1), order='F')
    print(yyvec.shape)

    Bvec = spla.inv(xxblkdiag.T@xxblkdiag)@(xxblkdiag.T@yyvec)
    
    A = Bvec.reshape((2,2), order='F').T
    print(A)
    
    
    
    
#     A = np.random.randn(2,2)
# A = np.array([[0, 1],[-2, -1]])

# x = np.random.randn(2,8)
# y = A@x

# print(A)
# print(x)
# print(y)

# print(y.T)
# print(y.T.reshape((-1, 1), order="F"))

# yls = y.T.reshape((-1, 1), order="F")
# xls = spla.block_diag(x.T,x.T)

# print(xls)

# Bvec = spla.inv(xls.T@xls)@(xls.T@yls)

# print(Bvec)

# B = Bvec.reshape((2,2), order='F')
# print(B)
# A = B.T
# print(A)
    

In [ ]:
#######################################################
#######################################################
####         Load data from file                    ###
#######################################################
#######################################################

#Retrieving the data from the load profile
TimeResolutionOfData = 10 #resolution in minute
#Get the data from the Testpvnum folder
#Provide Your Directory - move testpvnum10 from github to drive C: 
FileDirectoryBase ='../Data Files/testpvnum10/'
QSTS_Time = list(range(1441)) #This can be changed based on the available data - for example, 1440 timesteps
QSTS_Data = np.zeros((len(QSTS_Time),4,TotalLoads)) #4 columns as there are four columns of data available in the .mat file

for node in range(TotalLoads):
    #This is created manually according to the naming of the folder
    FileDirectoryExtension = 'node_' + str(node+1) + '_pv_' +str(TimeResolutionOfData) + '_minute.csv'
    #The total file directory
    FileName = FileDirectoryBase + FileDirectoryExtension
    #Load the file
    MatFile = np.genfromtxt(FileName, delimiter=',')    
    QSTS_Data[:,:,node] = MatFile #Putting the loads to appropriate nodes according to the loadlist
    
Generation = QSTS_Data[:,1,:]*GenerationScalingFactor #solar generation
Load = QSTS_Data[:,3,:]*LoadScalingFactor #load demand
Generation = np.squeeze(Generation)/Sbase  #To convert to per unit, it should not be multiplied by 100
Load = np.squeeze(Load)/Sbase
print('Reading Data for Pecan Street is done.')

Generation[Generation <= 0] = 0
Load[Load <= 0] = 0

Generation_minutes = Generation
Load_minutes = Load

len(Load[:,0])

In [ ]:
fig = plt.figure(figsize=[16,10])
for k1 in range(0, 3):
    plt.plot(Load[:,k1])
    
fig = plt.figure(figsize=[16,10])
for k1 in range(0, 3):
    plt.plot(Generation[:,k1])

In [ ]:
############################################################
############################################################
#### Interpolate to change data from minutes to seconds ####
############################################################
############################################################

StartTime = 0
EndTime = StartTime + 86400
EndTime += 1

from scipy.interpolate import interp1d

print('Starting Interpolation...')

#interpolation for the whole period...
Time = list(range(StartTime,EndTime))
TotalTimeSteps = len(Time)
LoadSeconds = np.empty([3600*24 + 1, TotalLoads])
GenerationSeconds = np.empty([3600*24 + 1, TotalLoads])
# Interpolate to get minutes to seconds
for node in range(TotalLoads): # i is node
    
    t_seconds = np.linspace(0,len(Load_minutes[:,node]),int(3600*24 + 1))
        
    f = interp1d(np.arange(len(Load_minutes[:,node])), Load_minutes[:,node], kind='cubic', fill_value="extrapolate")
    LoadSeconds[:,node] = f(t_seconds) #spline method in matlab equal to Cubic Spline -> cubic
    
#     print(np.arange(len(Load_minutes[:,node])))
#     print(t_seconds)
    
    f = interp1d(np.arange(len(Generation_minutes[:,node])), Generation_minutes[:,node], kind='cubic', fill_value="extrapolate")
    GenerationSeconds[:,node]= f(t_seconds)
    
# Initialization
# then we take out only the window we want...
# LoadSeconds = LoadSeconds[StartTime:EndTime,:]
# GenerationSeconds = GenerationSeconds[StartTime:EndTime,:]
Load = LoadSeconds
Generation = GenerationSeconds
timeList = list(range(TotalTimeSteps))
print('Finished Interpolation!')

print(len(t_seconds))
print(len(Load_minutes[:,node]))

Load.shape

In [ ]:
fig = plt.figure(figsize=[16,10])
for k1 in range(0, Load.shape[1]):
    plt.plot(t_seconds/60,Load[:,k1])
    
fig = plt.figure(figsize=[16,10])
for k1 in range(0, Load.shape[1]):
    plt.plot(t_seconds/60,Generation[:,k1])

In [ ]:

class inverter():
    
    def __init__(self):
        
        self.max_reactive_capacity = 1
        self.reactive_capacity = 1
    
        self.reactive_power = 0
    
        self.VBP = np.array([0.97, 0.99, 1.01, 1.03])
        
        self.wlp = 2*np.pi*0.10
        
        self.Vmeas = np.zeros(86401)
        self.Vlp = np.zeros(86401)
        
        self.reactive_power = np.zeros(86401)
        
        self.Ts = 1
        
        self.hackFlag = False
        
    def set_timesteps(self, startTimeStep, endTimeStep):
        self.startTimeStep = startTimeStep
        self.endTimeStep = endTimeStep
        
    def set_nodenum(self, nodenum):
        self.nodenum = nodenum
        
    def set_nodename(self, nodename):
        self.nodename = nodename    
            
    def set_VBP(self, VBP):
        self.VBP = VBP
        
    def measure_voltage(self, timeStep, Vmeas):
        self.Vmeas[kt] = Vmeas
        
    def lowpass(self, kt):
        self.Vlp[kt] = (1 - self.wlp*self.Ts)*self.Vlp[kt-1] + self.wlp*self.Ts*self.Vmeas[kt-1]
        
    def compute_reactive_power_output(self, kt, Vcomp):
        if Vcomp <= self.VBP[0]:
            self.reactive_power[kt] = -100
        elif self.VBP[0] <= Vcomp <= self.VBP[1]:
            self.reactive_power[kt] =100/(self.VBP[1] - self.VBP[0])*(Vcomp - self.VBP[1])
        elif self.VBP[1] <= Vcomp <= self.VBP[2]:
            self.reactive_power[kt] = 0
        elif self.VBP[2] <= Vcomp <= self.VBP[3]:
            self.reactive_power[kt] = 100/(self.VBP[3] - self.VBP[2])*(Vcomp - self.VBP[2])
        elif self.VBP[3] <= Vcomp:
            self.reactive_power[kt] = 100
        # print(self.reactive_power)
        
    

x1 = inverter()
x1.set_VBP(np.array([0.97, 0.99, 1.01, 1.03]))
print(x1.VBP)

x1.compute_reactive_power_output(0,1.02)
print(x1.reactive_power)

# x1.measure_voltage(0,1.02)
# x1.measure_voltage(1,1.02)
# x1.measure_voltage(2,1.02)
# x1.measure_voltage(3,1.01)
# x1.measure_voltage(4,1.01)
# x1.measure_voltage(5,1.01)


print(x1.Vmeas)
print(x1.Vlp)

In [ ]:
#########################################################
################### RESTART OPENDSS #####################
#########################################################
dss.run_command('Redirect feeder/03node_singlephase_balanced.dss')
dss.Solution.Solve()


if dss.Solution.Converged:
    print('Initial Model Converged. Proceeding to Next Step.')
    
else:
    print('Initial Solution Not Converged. Check Model for Convergence')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

    
#set Source Info for OpenDSS
#setSourceInfo(dss,['source'],'pu',[SlackBusVoltage]) 
dss.Vsources.PU(SlackBusVoltage)
############################################
############ INIT NODES ####################
############################################
""" 
 nodes' variable is a dictionary contains all the nodes in the grid,
 with KEY is the node's number, VALUE is a dataframe with 
 ROW is ['Voltage', 'Generation', 'P', 'Q'] - Voltage is Voltage for each timestep; 
                                              P,Q is P,Q injection at that node for that timestep.
 and 
 COLUMN is each timestep for the whole simulation. 
 nodes = {
    1: dataFrame1,
    2: dataFrame2
 }
"""

nodes = {}
features = ['Voltage', 'Generation', 'P', 'Q']

# for k1 in range(len(AllLoadNames)):
#     print(k1)
#     df = pd.DataFrame(columns=list(range(TotalTimeSteps)),index=features)
#     nodes[k1] = df
#     nodes[k1].loc['Generation'] = Generation[:,k1]
#     nodes[k1].loc['Load'] = Load[:,k1]
#     nodes[k1].loc['P'] = 0
#     nodes[k1].loc['Q'] = 0
#     nodes[k1].loc['Pinj'] = 0
#     nodes[k1].loc['Qinj'] = 0
    

    
################################################
############ INIT INVERTERS ####################
################################################
""" 
 inverters' variable is a dictionary contains all the inverters in the grid,
 with KEY is the node's number where we have inverters, VALUE is a list of inverters at that node.
 
 Each inverter has a dictionary:
     'device': Inverter_Object
     'controller: Controller_Object
     'info': contain scenario information in a dataframe (solar generation, sbar) for each timestep at that node
 }
"""

# we create inverters from node 5 to node (5+13)
offset = 5
numberofInverters = 13

# for k1 in range(len(AllLoadNames)):
#     inverters[k1] = []
#     if offset-1 < k1 < numberofInverters + offset:
#         # inverter device
#         inv = {}
#         inv['device'] = Inverter(timeList,lpf_meas=lpf_meas_vector[k1],lpf_output=lpf_output_vector[k1])
#         #controller: timeList, VBP is initial VBP, delayTimer is the delay control on VBP
#         inv['controller'] = AdaptiveInvController(timeList, 
#                                                   VBP = VBP_normal, 
#                                                   delayTimer=Delay_VBPCurveShift[k1],
#                                                   device=inv['device'], nk=adaptive_gain, threshold=ThreshHold_vqvp)
        
#         #prepare info
#         df = pd.DataFrame(columns=list(range(TotalTimeSteps)),index=features)
#         df.loc['Generation'] = Generation[:,k1]
#         df.loc['sbar'] = sbar[k1]
#         timeList = list(range(TotalTimeSteps))
#         inv['info'] = df
#         inverters[k1].append(inv)

# inverterLsist = {}


simStartTimeStep = 12*3600+1
simEndTimeStep = 12*3600+1 + 3600

simStartTimeStep = 6*3600+1
simEndTimeStep = 18*3600+1

simStartTimeStep = 12*3600+1
simEndTimeStep = 12*3600+1 + 1200

simStartTimeStep = 12*3600+1 - 300
simEndTimeStep = 12*3600+1 + 600

time = np.arange(simStartTimeStep,simEndTimeStep)

totalTimeSteps = simEndTimeStep - simStartTimeStep + 1

LoadBuses = dss.Loads.AllNames()
AllBuses = dss.Circuit.AllBusNames()

LoadBusVoltage = np.zeros((TotalTimeSteps,len(LoadBuses)))
AllBusVoltage = np.zeros((TotalTimeSteps,len(AllBuses)))


##################################################
# Set up loads
##################################################

Nload = 25
Load_factored = Load/1.25

load_indexes = np.zeros((len(LoadBuses), Nload)).astype(int)
for k1 in range(len(LoadBuses)):
    
    load_indexes[k1,:] = np.random.randint(0, Load.shape[1], size=Nload).astype(int)
#     load_indexes[k1,:] = load_indexes[k1,:].astype(int)
#     print(load_indexes[k1,:])

# print(load_indexes)
# print(Load_factored[load_indexes[0,:],0])


##################################################
# Set up generation
##################################################

Ngen = 15
Generation_factored = Generation/1.5

generation_indexes = np.zeros((len(LoadBuses), Ngen)).astype(int)
for k1 in range(len(LoadBuses)):
    
    generation_indexes[k1,:] = np.random.randint(0, Load.shape[1], size=Ngen).astype(int)
#     generation_indexes[k1,:] = generation_indexes[k1,:].astype(int)
#     print(generation_indexes[k1,:])

# print(generation_indexes)
# print(Generation_factored[generation_indexes[0,:],0])


##################################################
# Set up inverters
##################################################
       
inverters = []
for k1 in range(len(LoadBuses)):
    if LoadBuses[k1] == 's702a' or LoadBuses[k1] == 's702b' or LoadBuses[k1] == 's702c':
#         for k2 in range(generation_indexes.shape[1]):
        for k2 in range(0,1):

    #         inverters[k1] = []
            tempinverter = inverter()
            tempinverter.set_timesteps(simStartTimeStep, simEndTimeStep)
            tempinverter.set_nodenum(k1)
            tempinverter.set_nodename(LoadBuses[k1])
            tempinverter.set_VBP(np.array([0.95, 0.97, 1.03, 1.05]))
            tempinverter.set_VBP(np.array([0.95, 0.99, 1.01, 1.05]))

            inverters.append(tempinverter)
            
#     if LoadBuses[k1] == 's701a' or LoadBuses[k1] == 's701b' or LoadBuses[k1] == 's701c':
# #         for k2 in range(generation_indexes.shape[1]):
#         for k2 in range(0,1):

#     #         inverters[k1] = []
#             tempinverter = inverter()
#             tempinverter.set_timesteps(simStartTimeStep, simEndTimeStep)
#             tempinverter.set_nodenum(k1)
#             tempinverter.set_nodename(LoadBuses[k1])
#             tempinverter.set_VBP(np.array([0.95, 0.97, 1.03, 1.05]))
#             tempinverter.set_VBP(np.array([0.95, 0.99, 1.01, 1.05]))

#             inverters.append(tempinverter)

    
##################################################
# ?
##################################################

LoadBuses = dss.Loads.AllNames()
AllBuses = dss.Circuit.AllBusNames()

LoadBusVoltage = np.zeros((TotalTimeSteps,len(LoadBuses)))
# LoadBusVoltage0 = np.zeros((TotalTimeSteps,len(LoadBuses)))
# LoadBusVoltage1 = np.zeros((TotalTimeSteps,len(LoadBuses)))
# LoadBusVoltage2 = np.zeros((TotalTimeSteps,len(LoadBuses)))

AllBusVoltage = np.zeros((TotalTimeSteps,len(AllBuses)))

LoadBusPinj = np.zeros((TotalTimeSteps,len(LoadBuses))) 
LoadBusQinj = np.zeros((TotalTimeSteps,len(LoadBuses)))

kWloadTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
kVArloadTS = np.zeros((TotalTimeSteps,len(LoadBuses)))

kWtotalTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
kVArtotalTS = np.zeros((TotalTimeSteps,len(LoadBuses)))

PloadTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
PinjTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
PtotalTS = np.zeros((TotalTimeSteps,len(LoadBuses)))

QloadTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
QinjTS = np.zeros((TotalTimeSteps,len(LoadBuses)))
QtotalTS = np.zeros((TotalTimeSteps,len(LoadBuses)))


PloadNominal = np.zeros(len(LoadBuses))
QloadNominal = np.zeros(len(LoadBuses))
lf = np.zeros(len(LoadBuses))
for k1 in range(len(LoadBuses)):
    dss.Loads.Name(LoadBuses[k1])
    PloadNominal[k1] = dss.Loads.kW()
    QloadNominal[k1] = dss.Loads.kvar()
    lf[k1] = 1 + 0.2*(np.random.rand()-0.5)
    
print(LoadBuses)
print(PloadNominal)
print(QloadNominal)
print(lf)


#for each timestep in the simulation
# for kt in range(simStartTimeStep,simEndTimeStep):
for kt in range(simStartTimeStep,simEndTimeStep+1):
       
    if (kt - 1) % 60 == 0:
        hh = int((kt-1)/3600)
        mm = int(((kt-1) - 3600*int((kt-1)/3600))/60)
        if mm <= 9:
            print(str(hh) + '.0' + str(mm))
        else:
            print(str(hh) + '.' + str(mm))
    
#     print(kt)
    ####################################################
    ################ run the simulation ################
    ####################################################
    #for the first steps, we just initialize voltage value, no pq injection
    if kt == simStartTimeStep:
        for k1 in range(len(LoadBuses)):
            nodeName = LoadBuses[k1]
            dss.Loads.Name(nodeName)
            
#             dss.Loads.kW(1*Load_factored[kt, node])
#             dss.Loads.kvar(1*pf_converted*Load_factored[kt, node])
            
#             kWload = np.sum(Load_factored[kt,load_indexes[k1,:]])/25
#             kWloadTS[kt,k1] = kWload
#             kWtotalTS[kt,k1] = kWload
#             dss.Loads.kW(kWload)
            
            Pload = PloadNominal[k1]
            PloadTS[kt,k1] = Pload
            
            Ptotal = Pload
            PtotalTS[kt,k1] = Pload
            dss.Loads.kW(Ptotal)
            
            print(LoadBuses[k1], dss.Loads.kW())
            
#             kVArload = pf_converted*kWload
#             kVArloadTS[kt,k1] = kVArload
#             kVArtotalTS[kt,k1] = kVArload
#             dss.Loads.kvar(kVArload)
            
            Qload = QloadNominal[k1]
            QloadTS[kt,k1] = Qload
            
            Qtotal = Qload
            QtotalTS[kt,k1] = Qload
            dss.Loads.kvar(Qtotal)
            
            print(LoadBuses[k1], dss.Loads.kvar())

            
    #otherwise, we add Active Power (P) and Reactive Power (Q) which we injected at last timestep
    #to the grid at that node
    else:
        for k1 in range(len(LoadBuses)):
            nodeName = LoadBuses[k1]
            dss.Loads.Name(nodeName)
            
#             dss.Loads.kW(1*Load_factored[kt, node])
#             dss.Loads.kvar(1*pf_converted*Load_factored[kt, node])
            
#             kWload = np.sum(Load_factored[kt,load_indexes[k1,:]])/25
#             kWloadTS[kt,k1] = kWload
#             kWtotalTS[kt,k1] = kWload
#             dss.Loads.kW(kWload)

            Pload =  PloadNominal[k1] + 25*np.sin(2*np.pi*1/60*kt)
            PloadTS[kt,k1] = Pload
        
            Pinj = 0*-np.sum(Generation_factored[kt,generation_indexes[k1,:]])/Ngen
            PinjTS[kt,k1] = Pinj
            
            Ptotal = Pload + Pinj
            PtotalTS[kt,k1] = Ptotal
            dss.Loads.kW(Ptotal)
            
#             kVArload = pf_converted*kWload
#             kVArloadTS[kt,k1] = kVArload
            
#             kVArtotal = kVArload
#             for k2 in range(len(inverters)):
#                 if inverters[k2].nodename == LoadBuses[k2]:
#                     kVArtotal = kVArtotal + inverters[k2].reactive_power[kt-1]/100*1
#             kVArtotalTS[kt,k1] = kVArtotal
#             dss.Loads.kvar(kVArtotal)
            
            Qload = QloadNominal[k1] + 15*np.sin(2*np.pi*1/60*kt)
            QloadTS[kt,k1] = Qload
            
            Qinj = 0
            for k2 in range(len(inverters)):
                if inverters[k2].nodename == LoadBuses[k1]:
#                     print(inverters[k2].nodename, LoadBuses[k2])
                    Qinj = Qinj + inverters[k2].reactive_power[kt-1]/100*400
            Qinj = Qinj
            QinjTS[kt,k1] = Qinj
            
            Qtotal = Qload + 1*Qinj
            QtotalTS[kt,k1] = Qtotal
            dss.Loads.kvar(Qtotal)
            
            #setLoadInfo(DSSObj, [nodeName], 'kw', [Load[kt, node] + nodes[node].at['P', kt-1]])
            #setLoadInfo(DSSObj, [nodeName], 'kvar', [pf_converted*Load[kt, node] + nodes[node].at['Q', kt-1]])

    #solve() openDSS with new values of Load
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Solution Not Converged at Step:', kt)
        
    dss.Monitors.SampleAll()
    #get the voltage info
    #(voltage[0]+voltage[2]+voltage[4])/(DSSCircuit.ActiveElement.NumPhases*(Loads.kV*1000/(3**0.5)))
    nodeInfo = []
    
    for k1 in range(len(LoadBuses)):
        dss.Loads.Name(LoadBuses[k1])
        voltage = dss.CktElement.VoltagesMagAng()
#         print(dss.CktElement.NodeOrder())
#         print(voltage)
#         voltagePU = (voltage[0]+voltage[2]+voltage[4])/(dss.CktElement.NumPhases()*(dss.Loads.kV()*1000/(3**0.5)))

#         LoadBusVoltage[kt,k1] = voltagePU
        
#     for k1 in range(len(AllBuses)):
# #         dss.Loads.Name(LoadBuses[k1])
#         dss.Circuit.SetActiveBus(AllBuses[k1])
#         tempVoltagePU = dss.Bus.puVmagAngle()
# #         print(tempVoltagePU)
# #         print(tempVoltage)
# #         print(tempVoltage[::2])
#         tempVoltagePU = tempVoltagePU[::2]
#         tempVoltagePU = np.sum(tempVoltagePU)/len(tempVoltagePU)
# #         print(tempVoltage)
#         LoadBusVoltage[kt,k1] = tempVoltagePU
        
    for k1 in range(len(AllBuses)):
        dss.Circuit.SetActiveBus(AllBuses[k1])
        tempVoltagePU = dss.Bus.puVmagAngle()[::2]
#         print(AllBuses[k1])
#         print(tempVoltage)
        tempVoltagePU = np.sum(tempVoltagePU)/len(tempVoltagePU)
#         tempVoltagePU = tempVoltage/(dss.Bus.kVBase()*1000)        
        AllBusVoltage[kt,k1] = tempVoltagePU
        
#         print(dss.Bus.VMagAngle())
        
#         if kt == simStartTimeStep + 20:
#             print(AllBuses[k1], tempVoltagePU)
        
#     for k1 in range(len(LoadBuses)):
        
#         inverters[k1].measure_voltage(kt,LoadBusVoltage[kt,k1])
#         inverters[k1].lowpass(kt)
    
    # Iterate through inverters
    for k1 in range(len(inverters)):
        
        AllBuses.index(inverters[k1].nodename[1:4])
        
        inverters[k1].measure_voltage(kt,AllBusVoltage[kt,AllBuses.index(inverters[k1].nodename[1:4])])
        inverters[k1].lowpass(kt)
        
        if kt >= simStartTimeStep + 10:            
            inverters[k1].compute_reactive_power_output(kt, inverters[k1].Vlp[kt])
        
#         inverters[k1].compute_reactive_power_output(kt, inverters[k1].Vlp[kt])
        
    if kt == 12*3600+1+60:
        for k1 in range(len(inverters)):
            inverters[k1].set_VBP(inverters[k1].VBP)
            if np.random.rand() <= 2:
                inverters[k1].hackFlag = True
                inverters[k1].set_VBP(np.array([0.99, 0.995, 1.005, 1.01]))


                

In [ ]:
font = {'weight':'bold', 'size':12}

plt.rc('font', **font)

plotStartTimeStep = 12*3600 + 1
plotEndTimeStep = 12*3600 + 1 + 300

time = np.arange(plotStartTimeStep,plotEndTimeStep)

timehh = ((time-1)/3600).astype(int)
timemm = ((time-1) - 3600*timehh)/60

timeplot = timehh.astype(float) + timemm/100

fig = plt.figure(figsize=[12,6])
# for k1 in range(0,LoadBusVoltage.shape[1]):
for k1 in range(len(LoadBuses)):
#     plt.plot(time/3600,LoadBusVoltage[plotStartTimeStep:plotEndTimeStep,k1])
    plt.plot(timeplot,LoadBusVoltage[plotStartTimeStep:plotEndTimeStep,k1] - np.mean(LoadBusVoltage[plotStartTimeStep:plotEndTimeStep,k1]))
#     print(LoadBuses[k1])
plt.title('Node Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.show()


fig = plt.figure(figsize=[12,6])
# for k1 in range(0,LoadBusVoltage.shape[1]):
for k1 in range(len(AllBuses)):
    plt.plot(time/3600,AllBusVoltage[plotStartTimeStep:plotEndTimeStep,k1],label=AllBuses[k1])
#     print(LoadBuses[k1])
plt.title('Node Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.legend()
plt.show()

fig = plt.figure(figsize=[12,6])
for k1 in range(0,len(inverters)):
#     plt.plot(inverters[k1].Vmeas[plotStartTimeStep:plotEndTimeStep])
    plt.plot(time/3600,inverters[k1].Vlp[plotStartTimeStep:plotEndTimeStep])
# plt.ylim([0.85, 1.05])
plt.title('Inverter Low-pass Filter Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.show()

fig = plt.figure(figsize=[12,6])
for k1 in range(0,len(inverters)):
    plt.plot(time/3600,inverters[k1].reactive_power[plotStartTimeStep:plotEndTimeStep])
# plt.ylim([-100, 100])
plt.title('Inverter Reactive Power Output')
plt.xlabel('Time')
plt.ylabel('Percent Capacity [%]')
plt.show()

# fig = plt.figure(figsize=[12,6])
# plt.plot(time/3600,PloadTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.plot(time/3600,QloadTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.title('Load Demands')
# plt.xlabel('Time')
# plt.ylabel('Percent Capacity [%]')
# plt.show()

fig = plt.figure(figsize=[12,6])
plt.subplot(2,1,1)
plt.plot(time/3600,PloadTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Active Power Demand')
# plt.xlabel('Time')
plt.ylabel('Active Power Demand [kW]')
plt.subplot(2,1,2)
plt.plot(time/3600,QloadTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Reactive Power Demand')
plt.xlabel('Time')
plt.ylabel('Reactive Power Demand [kVAr]')
plt.show()

# fig = plt.figure(figsize=[12,6])
# plt.plot(time/3600,PinjTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.plot(time/3600,QinjTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.title('Injection')
# plt.show()

fig = plt.figure(figsize=[12,6])
plt.subplot(2,1,1)
plt.plot(time/3600,PinjTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Active Power Injection')
# plt.xlabel('Time')
plt.ylabel('Active Power Injection [kW]')
plt.subplot(2,1,2)
plt.plot(time/3600,QinjTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Reactive Power Injection')
plt.xlabel('Time')
plt.ylabel('Reactive Power Injection [kVAr]')
plt.show()

# fig = plt.figure(figsize=[12,6])
# plt.plot(time/3600,PtotalTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.plot(time/3600,QtotalTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.title('Total')
# plt.show()

fig = plt.figure(figsize=[12,6])
plt.subplot(2,1,1)
plt.plot(time/3600,PtotalTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Total Active Power')
# plt.xlabel('Time')
plt.ylabel('Total Active Power [kW]')
plt.subplot(2,1,2)
plt.plot(time/3600,QtotalTS[plotStartTimeStep:plotEndTimeStep,:])
plt.title('Node Total Reactive Power')
plt.xlabel('Time')
plt.ylabel('Total Reactive Power [kVAr]')
plt.show()

# print(timeplot)


In [ ]:
dss.Circuit.SetActiveBus(AllBuses[2])
dss.Bus.Nodes()

In [ ]:
inverters[0].nodename

In [ ]:
print(AllBuses)

AllBusVoltage[simStartTimeStep,:]

In [ ]:
Zbase = ((4.8e3/np.sqrt(3))**2)/(2500e3/3)

print(Zbase)

dss.Lines.Name('line_799_701')
dss.Lines.Name()
dss.Lines.XMatrix()

Xmat = np.array(dss.Lines.XMatrix())
Xmat = Xmat*dss.Lines.Length()
Xmatpu = Xmat/Zbase

print(Xmatpu)


dss.Lines.Name('line_701_702')
dss.Lines.Name()
dss.Lines.XMatrix()

Xmat = np.array(dss.Lines.XMatrix())
Xmat = Xmat*dss.Lines.Length()

Xmatpu = Xmat/Zbase

print(Xmatpu)

x01 = 0.08
x12 = 0.00759549
x23 = 0.00596788

print(Xmat)

Xmat = np.array([[x01, x01, x01],[x01, x01+x12, x01+x12],[x01, x01+x12,x01+x12+x23]])
spla.inv(Xmat)

Xmat.shape

In [ ]:
Ns = 20

# for kt in range(12*3600+1+60,simEndTimeStep-Ns):
for kt in range(12*3600+1+60,12*3600+1+60+2):
    
    print(kt)
    
    xx = AllBusVoltage[kt:kt+Ns,1:].T
    print(xx.shape)
    
    yy = AllBusVoltage[kt+1:kt+1+Ns,1:].T
    print(yy.shape)

    xxblkdiag = spla.block_diag(xx.T,xx.T)
    for k1 in range(1,xx.shape[0]-1):
        xxblkdiag = spla.block_diag(xxblkdiag,xx.T)
    print(xxblkdiag.shape)

    yyvec = yy.T.reshape((-1,1), order='F')
    print(yyvec.shape)

    Bvec = spla.inv(xxblkdiag.T@xxblkdiag)@(xxblkdiag.T@yyvec)
#     print(Bvec)
    
    Ahat = Bvec.reshape((xx.shape[0],xx.shape[0]), order='F').T
    print(Ahat)
    
    A = spla.inv(Xmat)@Ahat
    print(A)
    
    U, s, Vh = spla.svd(A)
    print(U)
    print(s)
    print(Vh)
    
#     print(np.diag(s[[0]]))
        
    print(U[:,[0]]@np.diag(s[[0]])@Vh[[0],:])
    

In [ ]:
Ns = 20

# for kt in range(12*3600+1+60,simEndTimeStep-Ns):
for kt in range(12*3600+1+60,12*3600+1+60+2):
    
    print(kt)
    
    xx = AllBusVoltage[kt:kt+Ns,1:].T - 1
    print(xx.shape)
    
    yy = AllBusVoltage[kt+1:kt+1+Ns,1:].T
    print(yy.shape)
    
    yy = QinjTS[kt+1:kt+1+Ns,:].T
    print(yy.shape)

    xxblkdiag = xx.T
    for k1 in range(1,yy.shape[0]):
        xxblkdiag = spla.block_diag(xxblkdiag,xx.T)
    print(xxblkdiag.shape)

    yyvec = yy.T.reshape((-1,1), order='F')
    print(yyvec.shape)

    Bvec = spla.inv(xxblkdiag.T@xxblkdiag)@(xxblkdiag.T@yyvec)
#     print(Bvec)
    
    Ahat = Bvec.reshape((yy.shape[0],xx.shape[0]), order='F').T
    print(Ahat)
    
#     A = spla.inv(Xmat)@Ahat
#     print(A)
    
#     U, s, Vh = spla.svd(A)
#     print(U)
#     print(s)
#     print(Vh)
    
# #     print(np.diag(s[[0]]))
        
#     print(U[:,[0]]@np.diag(s[[0]])@Vh[[0],:])
    

In [ ]:
dir(dss.Lines.AllNames)

dss.Lines.AllNames()

In [ ]:
Zbase = ((4.8e3/np.sqrt(3))**2)/(2500e3/3)

print(Zbase)

dss.Lines.Name('line_799_701')
dss.Lines.Name()
dss.Lines.XMatrix()

Xmat = np.array(dss.Lines.XMatrix())
Xmat = Xmat*dss.Lines.Length()
Xmatpu = Xmat/Zbase

print(Xmatpu)


dss.Lines.Name('line_701_702')
dss.Lines.Name()
dss.Lines.XMatrix()

Xmat = np.array(dss.Lines.XMatrix())
Xmat = Xmat*dss.Lines.Length()

Xmatpu = Xmat/Zbase

print(Xmatpu)

x01 = 0.1
x12 = 0.00759549
x23 = 0.00596788

X = np.array([[x01, x01, x01],[x01, x01+x12, x01+x12],[x01, x01+x12,x01+x12+x23]])
spla.inv(X)

print(X)
print(spla.inv(X))

In [ ]:
QinjTS.shape

In [ ]:
QtotalTS.shape